In [1]:
import numpy as np
from scipy.stats import t
import pandas as pd  # Удобно для вывода таблиц

# 1. Исходные данные (из колонок 3 и 14)
X = np.array([13.26, 10.10, 13.72, 12.85, 10.63, 9.12, 28.89, 23.39, 14.68, 10.05,
              13.66, 9.68, 10.03, 9.13, 9.37, 9.86, 12.62, 5.02, 21.18, 25.17,
              9.28, 11.42, 10.31, 8.65, 10.94, 9.87, 6.14, 12.93, 19.73, 13.22,
              17.29, 7.11, 22.49, 12.14, 15.28, 31.34, 11.56, 30.14, 19.71, 23.56])

Y = np.array([6.4, 7.8, 9.76, 7.9, 5.35, 9.9, 4.5, 4.86, 3.46, 9.6,
              9.26, 5.65, 4.28, 8.85, 8.52, 7.19, 4.32, 5.46, 6.2, 4.25,
              5.65, 6.67, 5.91, 11.99, 8.3, 1.03, 5.84, 5.82, 4.8, 5.01,
              4.12, 5.1, 9.49, 4.19, 5.01, 11.44, 7.67, 4.66, 4.3, 6.62])

N = len(X)
alpha = 0.05

print(f"=== 1. Исходные данные и параметры ===")
print(f"Объем выборки (n): {N}")
print(f"Уровень значимости (alpha): {alpha}\n")


# 2. Функция для ранжирования (с учетом связанных рангов)
def rank_data(data):
    # Создаем Series для удобства обработки связанных рангов
    s = pd.Series(data)
    # method='average' гарантирует, что связанным значениям присваивается средний ранг
    ranks = s.rank(method='average', ascending=True)
    return ranks.to_numpy()


# 3. Ранжирование данных
R_X = rank_data(X)
R_Y = rank_data(Y)

print("-" * 20)
print(f"=== 2. Ранжирование данных ===")
print(f"Пример рангов X (первые 5): {R_X[:5]}")
print(f"Пример рангов Y (первые 5): {R_Y[:5]}\n")

# 4. Расчет разности рангов (D) и квадрата разности (D^2)
D = R_X - R_Y
D_sq = D ** 2
sum_D_sq = np.sum(D_sq)

# Создание DataFrame для промежуточного вывода
df_results = pd.DataFrame({
    'X': X, 'Y': Y,
    'Rank(X)': R_X, 'Rank(Y)': R_Y,
    'D': D, 'D^2': D_sq
})

print(f"=== 3. Расчет Sum(D^2) ===")
# Выводим первые и последние строки для примера (to_string вместо to_markdown для работы без tabulate)
print(df_results[['X', 'Y', 'Rank(X)', 'Rank(Y)', 'D', 'D^2']].head(5).to_string(index=False))
print("...")
print(df_results[['X', 'Y', 'Rank(X)', 'Rank(Y)', 'D', 'D^2']].tail(5).to_string(index=False, header=False))

print(f"\nСумма квадратов разностей рангов (Sum D^2): {sum_D_sq:.2f}")

# 5. Расчет коэффициента корреляции Спирмена (r_s)
# Формула Спирмена
rs = 1 - (6 * sum_D_sq) / (N * (N ** 2 - 1))

print("-" * 20)
print(f"=== 4. Расчет коэффициента Спирмена (r_s) ===")
print(f"Формула: r_s = 1 - (6 * Sum(D^2)) / (n * (n^2 - 1))")
print(f"Расчет: r_s = 1 - (6 * {sum_D_sq:.2f}) / (40 * (40^2 - 1)) = {rs:.4f}")

print(f"\nКоэффициент корреляции Спирмена (r_s): {rs:.4f}")
# Значение r_s, полученное через scipy.stats.spearmanr: -0.11903 (для проверки)

# 6. Расчет t-статистики (t_st)
# Формула t-статистики: t_st = r_s * sqrt((n - 2) / (1 - r_s^2))
t_stat = abs(rs) * np.sqrt((N - 2) / (1 - rs ** 2))

print("-" * 20)
print(f"=== 5. Проверка значимости (t-критерий) ===")
print(f"Степени свободы (df = n - 2): {N - 2}")
print(f"Формула t-статистики: t_st = r_s * sqrt((n - 2) / (1 - r_s^2))")
print(f"Расчет t_st: {rs:.4f} * sqrt(({N} - 2) / (1 - {rs ** 2:.4f})) = {t_stat:.4f}")

# 7. Определение критического значения (t_kr)
# t_kr = t(alpha/2; n-2) для двустороннего теста
df = N - 2
prob_for_critical_t = 1 - alpha / 2  # Для t-распределения
t_critical = t.ppf(prob_for_critical_t, df)

print(f"\nКритическое значение t_kr:")
print(f"Используем t({alpha}/2; {df}): t(0.025; 38)")
print(f"t_kr = {t_critical:.4f}")

# 8. Сравнение и вывод
print("-" * 20)
print(f"=== 6. Вывод ===")
print(f"Рассчитанная t-статистика (|t_st|): |{t_stat:.4f}| = {abs(t_stat):.4f}")
print(f"Критическое значение (t_kr): {t_critical:.4f}\n")

# Проверка условия |t_st| > t_kr
if abs(t_stat) > t_critical:
    print(f"Результат: Так как |t_st| ({abs(t_stat):.4f}) > t_kr ({t_critical:.4f}), нулевая гипотеза ОТВЕРГАЕТСЯ.")
    print(f"Следовательно, с вероятностью 95% коэффициент корреляции значимо отличается от нуля.")
else:
    print(f"Результат: Так как |t_st| ({abs(t_stat):.4f}) <= t_kr ({t_critical:.4f}), нулевая гипотеза НЕ ОТВЕРГАЕТСЯ.")
    print(f"Следовательно, коэффициент корреляции r_s={rs:.4f} НЕ ЯВЛЯЕТСЯ значимым на уровне alpha=0.05.")

=== 1. Исходные данные и параметры ===
Объем выборки (n): 40
Уровень значимости (alpha): 0.05

--------------------
=== 2. Ранжирование данных ===
Пример рангов X (первые 5): [25. 14. 27. 22. 16.]
Пример рангов Y (первые 5): [24. 29. 37. 30. 16.]

=== 3. Расчет Sum(D^2) ===
    X    Y  Rank(X)  Rank(Y)     D   D^2
13.26 6.40     25.0     24.0   1.0   1.0
10.10 7.80     14.0     29.0 -15.0 225.0
13.72 9.76     27.0     37.0 -10.0 100.0
12.85 7.90     22.0     30.0  -8.0  64.0
10.63 5.35     16.0     16.0   0.0   0.0
...
31.34 11.44 40.0 39.0  1.0   1.0
11.56  7.67 19.0 28.0 -9.0  81.0
30.14  4.66 39.0 10.0 29.0 841.0
19.71  4.30 31.0  7.0 24.0 576.0
23.56  6.62 36.0 25.0 11.0 121.0

Сумма квадратов разностей рангов (Sum D^2): 13136.00
--------------------
=== 4. Расчет коэффициента Спирмена (r_s) ===
Формула: r_s = 1 - (6 * Sum(D^2)) / (n * (n^2 - 1))
Расчет: r_s = 1 - (6 * 13136.00) / (40 * (40^2 - 1)) = -0.2323

Коэффициент корреляции Спирмена (r_s): -0.2323
--------------------
=== 5

In [8]:
import pandas as pd
import numpy as np  # Добавлен импорт numpy, так как используется np.array
from scipy.stats import norm

# --- 1. Исходные данные и параметры ---
# Переменная X (Колонка 3)
X = np.array([13.26, 10.10, 13.72, 12.85, 10.63, 9.12, 28.89, 23.39, 14.68, 10.05,
              13.66, 9.68, 10.03, 9.13, 9.37, 9.86, 12.62, 5.02, 21.18, 25.17,
              9.28, 11.42, 10.31, 8.65, 10.94, 9.87, 6.14, 12.93, 19.73, 13.22,
              17.29, 7.11, 22.49, 12.14, 15.28, 31.34, 11.56, 30.14, 19.71, 23.56])

# Переменная Y (Колонка 14)
Y = np.array([6.4, 7.8, 9.76, 7.9, 5.35, 9.9, 4.5, 4.86, 3.46, 9.6,
              9.26, 5.65, 4.28, 8.85, 8.52, 7.19, 4.32, 5.46, 6.2, 4.25,
              5.65, 6.67, 5.91, 11.99, 8.3, 1.03, 5.84, 5.82, 4.8, 5.01,
              4.12, 5.1, 9.49, 4.19, 5.01, 11.44, 7.67, 4.66, 4.3, 6.62])

N = len(X)
alpha = 0.05

print(f"=== 1. Исходные данные и параметры ===")
print(f"Объем выборки (n): {N}")
print(f"Уровень значимости (alpha): {alpha}\n")


# --- 2. Ранжирование данных ---
# Функция для ранжирования (с учетом связанных рангов)
def rank_data(data):
    s = pd.Series(data)
    # method='average' для учета связанных рангов
    ranks = s.rank(method='average', ascending=True)
    return ranks.to_numpy()


R_X = rank_data(X)
R_Y = rank_data(Y)

# Объединяем в DataFrame для удобства сортировки по R_X
df = pd.DataFrame({'X': X, 'Y': Y, 'R_X': R_X, 'R_Y': R_Y})
# Сортируем по рангу X (восходящий порядок)
df_sorted = df.sort_values(by='R_X').reset_index(drop=True)

# Получаем ранги Y, соответствующие отсортированным рангам X (R_X от 1 до N)
R_Y_sorted = df_sorted['R_Y'].values

print("-" * 20)
print(f"=== 2. Подготовка рангов для подсчета P и Q ===")
print(f"Ранги Y, отсортированные по рангам X (R_X):")
print(R_Y_sorted)

# --- 3. Расчет P (Согласованные пары) и Q (Несогласованные пары) ---

P = 0  # Количество согласованных пар
Q = 0  # Количество несогласованных пар

print("\n" + "-" * 20)
print(f"=== 3. Подсчет согласованных (P) и несогласованных (Q) пар ===")

# Подсчет P и Q (перебираем все пары)
# Идем по отсортированному ряду R_Y_sorted
for i in range(N):
    # Текущий ранг R_Y_i
    rank_i = R_Y_sorted[i]

    # Считаем, сколько последующих рангов (R_Y_j, где j > i)
    # больше (P) или меньше (Q) текущего ранга R_Y_i.

    # Сумма количества рангов, больших чем rank_i, среди последующих рангов
    p_i = np.sum(R_Y_sorted[i + 1:] > rank_i)

    # Сумма количества рангов, меньших чем rank_i, среди последующих рангов
    q_i = np.sum(R_Y_sorted[i + 1:] < rank_i)

    P += p_i
    Q += q_i

    print(f"Для R_Y[{i + 1}] = {rank_i:.1f}: P+ = {p_i}, Q- = {q_i}")

# Расчет S
S = P - Q

print("\nСумма P (согласованные):", P)
print("Сумма Q (несогласованные):", Q)
print(f"Разность S = P - Q: S = {P} - {Q} = {S}")

# --- 4. Расчет коэффициента Кендалла (tau) ---
# Формула (без поправки на связанные ранги)
tau = (2 * S) / (N * (N - 1))

print("-" * 20)
print(f"=== 4. Расчет коэффициента Кендалла (r_K) ===")
print(f"Формула: r_K = (2 * S) / (n * (n - 1))")
print(f"Расчет: r_K = (2 * {S}) / (40 * (40 - 1)) = {tau:.4f}")
print(f"\nКоэффициент Кендалла (r_K): {tau:.4f}")

# Проверка значимости для N > 10
u_critical = norm.ppf(1 - alpha / 2)

# Расчет стандартной ошибки (denominator)
# sigma_S^2 = n(n-1)(2n+5) / 18
sigma_S_sq = (N * (N - 1) * (2 * N + 5)) / 18
SE_S = np.sqrt(sigma_S_sq)

# Расчет u-статистики
u_stat = S / SE_S

# Расчет критического значения для тау (u-критерия)
tau_critical_example = u_critical * np.sqrt((2 * N + 5) / (9 * N * (N - 1)))

print("-" * 20)
print(f"=== 5. Проверка значимости (u-критерий) ===")
print(f"Критическое значение u_kr для alpha=0.05: u_0.975 = {u_critical:.2f}")



print(f"\nРассчитанная u-статистика:")
print(f"Стандартная ошибка S (SE_S): sqrt((n(n-1)(2n+5))/18) = {SE_S:.2f}")
print(f"u_st = S / SE_S = {S} / {SE_S:.2f} = {u_stat:.4f}")

# --- 6. Сравнение и вывод ---
print("-" * 20)
print(f"=== 6. Вывод ===")

# Сравнение по u-статистике
print("Сравнение u-статистик:")
print(f"Рассчитанная u_st (|u_st|): {abs(u_stat):.4f}")
print(f"Критическое значение u_kr: {u_critical:.2f}")

if abs(u_stat) > u_critical:
    print(f"Результат (по u): Так как |u_st| ({abs(u_stat):.4f}) > u_kr ({u_critical:.2f}), нулевая гипотеза ОТВЕРГАЕТСЯ.")
else:
    print(f"Результат (по u): Так как |u_st| ({abs(u_stat):.4f}) <= u_kr ({u_critical:.2f}), нулевая гипотеза НЕ ОТВЕРГАЕТСЯ.")
print("-" * 20)

=== 1. Исходные данные и параметры ===
Объем выборки (n): 40
Уровень значимости (alpha): 0.05

--------------------
=== 2. Подготовка рангов для подсчета P и Q ===
Ранги Y, отсортированные по рангам X (R_X):
[17.  21.  15.  40.  38.  33.  18.5 32.  18.5 27.   1.   6.  36.  29.
 22.  16.  31.  26.  28.   4.   8.  30.  20.  13.5 24.  34.  37.   2.
 13.5  3.   7.  11.  23.  35.  12.  25.   5.   9.  10.  39. ]

--------------------
=== 3. Подсчет согласованных (P) и несогласованных (Q) пар ===
Для R_Y[1] = 17.0: P+ = 23, Q- = 16
Для R_Y[2] = 21.0: P+ = 19, Q- = 19
Для R_Y[3] = 15.0: P+ = 23, Q- = 14
Для R_Y[4] = 40.0: P+ = 0, Q- = 36
Для R_Y[5] = 38.0: P+ = 1, Q- = 34
Для R_Y[6] = 33.0: P+ = 5, Q- = 29
Для R_Y[7] = 18.5: P+ = 17, Q- = 15
Для R_Y[8] = 32.0: P+ = 5, Q- = 27
Для R_Y[9] = 18.5: P+ = 16, Q- = 15
Для R_Y[10] = 27.0: P+ = 9, Q- = 21
Для R_Y[11] = 1.0: P+ = 29, Q- = 0
Для R_Y[12] = 6.0: P+ = 24, Q- = 4
Для R_Y[13] = 36.0: P+ = 2, Q- = 25
Для R_Y[14] = 29.0: P+ = 6, Q- = 20
Для R_Y

In [3]:
import pandas as pd
from scipy.stats import chi2

# --- 1. Исходные данные и параметры ---
# Данные из IMG_20251204_004050_069.jpg
# Эксперты: 2, 3, 7
data = {
    'Expert 2': [5, 5, 5, 5, 4, 5, 5, 3, 4, 4, 4, 4, 3],
    'Expert 3': [5, 5, 5, 5, 4, 5, 4, 5, 3, 5, 4, 4, 5],
    'Expert 7': [5, 5, 3, 5, 4, 5, 4, 5, 4, 2, 4, 5, 5]
}
competencies = [
    'Result', 'Organization', 'Influence', 'Communication',
    'Confidence', 'Conflict', 'Adaptability', 'Teamwork',
    'Client', 'Listening', 'Openness', 'Risks', 'Flexibility'
]
df = pd.DataFrame(data, index=competencies)

# Параметры
m = len(df.columns)  # Количество экспертов (переменных)
n = len(df.index)    # Количество компетенций (объектов)
alpha = 0.05         # Уровень значимости

print(f"=== 1. Исходные данные и параметры ===")
print(f"Количество экспертов (m): {m}")
print(f"Количество компетенций (n): {n}")
print(f"Уровень значимости (alpha): {alpha}\n")

# --- 2. Ранжирование компетенций (объектов) по каждому эксперту ---

# Ранжируем каждую колонку (эксперта) отдельно.
# Для ранжирования используется метод 'average' для учета одинаковых оценок.
df_ranks = df.apply(lambda col: col.rank(method='average', ascending=False), axis=0)
# ascending=False: более высокая оценка (5) получает меньший (лучший) ранг (1, 2, ...).

print("-" * 20)
print(f"=== 2. Ранжирование компетенций по экспертам (Rij) ===")
# Используем to_string вместо to_markdown
print(df_ranks.to_string())


# --- 3. Расчет суммы рангов (Sum_rij) и промежуточных значений ---

# 1. Сумма рангов по каждой компетенции (строке)
df_ranks['Sum_rij'] = df_ranks.sum(axis=1)
R_sum = df_ranks['Sum_rij'].values

# 2. Вычисление среднего значения суммы рангов (m(n+1)/2)
# Это константа, которая вычитается из суммы рангов
mean_sum_ranks = m * (n + 1) / 2

# 3. Разность между суммой рангов и средним значением
R_diff = R_sum - mean_sum_ranks

# 4. Квадрат разности
R_diff_sq = R_diff ** 2

# 5. Сумма квадратов разностей
S = R_diff_sq.sum()

print("-" * 20)
print(f"=== 3. Расчет промежуточных значений ===")

# Создание DataFrame для промежуточного вывода
df_intermediate = pd.DataFrame({
    'Sum_rij': R_sum,
    f'Diff (Sum - {mean_sum_ranks:.1f})': R_diff,
    f'Diff^2': R_diff_sq
}, index=competencies)

print(df_intermediate.to_string())

print(f"\nСреднее значение суммы рангов m(n+1)/2: {mean_sum_ranks:.2f}")
print(f"Сумма квадратов разностей S = Sum(Sum_rij - Mean)^2: {S:.2f}\n")


# --- 4. Расчет коэффициента конкордации Кендалла (W) ---

# Формула (2.8):
# W(m) = 12 * S / (m^2 * (n^3 - n))
W_hat = (12 * S) / (m**2 * (n**3 - n))

print("-" * 20)
print(f"=== 4. Расчет коэффициента конкордации (W) ===")
print(f"Формула: W = (12 * S) / (m^2 * (n^3 - n))")
print(f"Расчет: W = (12 * {S:.2f}) / ({m}^2 * ({n}^3 - {n})) = {W_hat:.4f}")


# --- 5. Проверка значимости (Хи-квадрат Chi^2) ---

# Для n > 7 используется критерий Хи-квадрат
# Хи-квадрат (Chi^2) = m * (n - 1) * W(m)

chi_sq_stat = m * (n - 1) * W_hat

# Определение критического значения Chi^2
# Степени свободы: df = n - 1
df_chi = n - 1
chi_sq_critical = chi2.ppf(1 - alpha, df_chi)

print("-" * 20)
print(f"=== 5. Проверка значимости (Chi^2-критерий) ===")

print(f"Рассчитанное значение Chi^2_st: m * (n-1) * W = {m} * ({n}-1) * {W_hat:.4f} = {chi_sq_stat:.4f}")

print(f"\nКритическое значение Chi^2_kr (alpha={alpha}; df={df_chi}):")
print(f"Chi^2_kr = {chi_sq_critical:.4f}")

# --- 6. Вывод ---

print("-" * 20)
print(f"=== 6. Вывод ===")

# Условие: Chi^2_st > Chi^2_kr
if chi_sq_stat > chi_sq_critical:
    print(f"Результат: Так как Chi^2_st ({chi_sq_stat:.4f}) > Chi^2_kr ({chi_sq_critical:.4f}), нулевая гипотеза об отсутствии согласованности ОТВЕРГАЕТСЯ.")
    print(f"Это означает, что коэффициент конкордации значимо отличается от нуля с вероятностью 95%.")
else:
    print(f"Результат: Так как Chi^2_st ({chi_sq_stat:.4f}) <= Chi^2_kr ({chi_sq_critical:.4f}), нулевая гипотеза об отсутствии согласованности НЕ ОТВЕРГАЕТСЯ.")
    print(f"Следовательно, нет достаточных оснований утверждать, что оценки экспертов значимо согласованы на уровне alpha=0.05.")

=== 1. Исходные данные и параметры ===
Количество экспертов (m): 3
Количество компетенций (n): 13
Уровень значимости (alpha): 0.05

--------------------
=== 2. Ранжирование компетенций по экспертам (Rij) ===
               Expert 2  Expert 3  Expert 7
Result              3.5       4.5       4.0
Organization        3.5       4.5       4.0
Influence           3.5       4.5      12.0
Communication       3.5       4.5       4.0
Confidence          9.0      10.5       9.5
Conflict            3.5       4.5       4.0
Adaptability        3.5      10.5       9.5
Teamwork           12.5       4.5       4.0
Client              9.0      13.0       9.5
Listening           9.0       4.5      13.0
Openness            9.0      10.5       9.5
Risks               9.0      10.5       4.0
Flexibility        12.5       4.5       4.0
--------------------
=== 3. Расчет промежуточных значений ===
               Sum_rij  Diff (Sum - 21.0)  Diff^2
Result            12.0               -9.0   81.00
Organization  